In [49]:
# Импортирование необходимых библиотек
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_absolute_error

# Загрузка модели
model = joblib.load('model.pkl')

# Загрузка данных
train_df = pd.read_excel('train.xlsx')
test_df = pd.read_excel('test.xlsx')

# Преобразование дат в datetime формат
train_df['dt'] = pd.to_datetime(train_df['dt'])
test_df['dt'] = pd.to_datetime(test_df['dt'])

# Создание дополнительных признаков на основе истории (лаги, скользящие средние)
num_lags = 5

# Для тренировки данных
train_df['diff_1'] = train_df['Цена на арматуру'].diff()
for lag in range(1, num_lags + 1):
    train_df[f'lag_{lag}'] = train_df['Цена на арматуру'].shift(lag)
train_df['rolling_mean_4'] = train_df['Цена на арматуру'].rolling(window=4).mean()
train_df['rolling_mean_8'] = train_df['Цена на арматуру'].rolling(window=8).mean()
train_df['year'] = train_df['dt'].dt.year
train_df['month'] = train_df['dt'].dt.month
train_df['week'] = train_df['dt'].dt.isocalendar().week

# Убираем пропущенные значения
train_df = train_df.dropna()

# Разделение на признаки (X) и целевую переменную (y)
X_train = train_df.drop(columns=['dt', 'Цена на арматуру'])
y_train = train_df['Цена на арматуру']

# Оценка модели на обучающих данных
y_train_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_train_pred)
print(f'Mean Absolute Error on training data: {mae}')

# Подготовка к прогнозированию на 52 недели
last_train_date = train_df['dt'].iloc[-1]
future_dates = pd.date_range(start=last_train_date + pd.Timedelta(weeks=1), periods=52, freq='W-MON')

history = train_df.set_index('dt')['Цена на арматуру'].copy()
forecast_df = pd.DataFrame(index=future_dates, columns=['Forecast'])

# Прогноз на 52 недели вперед
for date in forecast_df.index:
    lags = []
    for lag in range(1, num_lags + 1):
        if len(history) >= lag:
            lags.append(history.iloc[-lag])
        else:
            lags.append(np.nan)

    diff_1 = history.diff().iloc[-1] if len(history) >= 2 else np.nan
    rolling_4 = history.rolling(4).mean().iloc[-1] if len(history) >= 4 else np.nan
    rolling_8 = history.rolling(8).mean().iloc[-1] if len(history) >= 8 else np.nan

    features = pd.DataFrame({
        'diff_1': [diff_1],
        'lag_1': [lags[0]],
        'lag_2': [lags[1]],
        'lag_3': [lags[2]],
        'lag_4': [lags[3]],
        'lag_5': [lags[4]],
        'rolling_mean_4': [rolling_4],
        'rolling_mean_8': [rolling_8],
        'year': [date.year],
        'month': [date.month],
        'week': [date.isocalendar().week]
    })

    features = features.reindex(columns=X_train.columns)

    forecast = model.predict(features)[0]  # Прогноз на следующую неделю
    forecast_df.loc[date, 'Forecast'] = forecast
    history.loc[date] = forecast  # Обновляем history для следующего предсказания

# Сохранение результатов прогноза в Excel
forecast_df = forecast_df.reset_index().rename(columns={'index': 'dt'})
output_df = pd.concat([train_df[['dt', 'Цена на арматуру']], forecast_df], ignore_index=True)
output_df.to_excel('test_with_forecast.xlsx', index=False)

print("Прогноз на 52 недели успешно добавлен в test_with_forecast.xlsx.")

Mean Absolute Error on training data: 108.14092254638672
Прогноз на 52 недели успешно добавлен в test_with_forecast.xlsx.
